In [ ]:
from pathlib import Path
from torch.utils.data import DataLoader, Dataset
from torchvision.models import resnet18
import numpy as np
import cv2
import random
from matplotlib import pyplot as plt
import torch
import torchvision.models as models
import torch.nn as nn
from torch import optim
from torch.nn.modules import conv, MaxPool2d
from torchsummary import summary

In [ ]:
model = models.resnet18(pretrained=True)
#Change the stride and/or padding of some layers
model.maxpool = MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
model.layer3[0].conv1 = conv.Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
model.layer3[0].downsample[0] = nn.Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
model.fc = nn.Linear(model.fc.in_features, 100)

In [ ]:
data_dir=Path("C:/Users/sclab")/'Helperdata'
train_dir=data_dir/'facescrub_train'
test_dir=data_dir/'facescrub_test'

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

In [ ]:
label_to_name=dict()
idx=0
for dir in train_dir.iterdir():
    label_to_name[idx]=dir.name
    idx+=1

label_to_name

In [ ]:
class MyDataset(Dataset):
    def __init__(self, path:Path):
        self.mean = np.array(mean).reshape((1, 1, 3))
        self.std = np.array(std).reshape((1, 1, 3))
        self.images=[]
        for dir in path.iterdir():
            for img in dir.rglob('*'):
                img=str(img)
                image=cv2.imread(img)
                #print(type(image))
                self.images.append(image)           

    def __len__(self):
        return len(self.images)
    # Todo
    def checktype(self):
        print(type(self.images[0]))

    def __getitem__(self, index):
        img=self.images[index]
        img = img / 255.
        img = (img - self.mean) / self.std
        img = np.transpose(img, [2, 0, 1])
        if len(self.images)==4000:
            label=index//40
        else:
            label=index//10 
        img = torch.tensor(img, dtype=torch.float32)
        return img, label

train_set=MyDataset(train_dir)
test_set=MyDataset(test_dir)
print(len(train_set))
print(len(test_set))

In [ ]:
#show some images
def denorm(img):
    for i in range(img.shape[0]):
        img[i] = img[i] * std[i] + mean[i]
    img = torch.clamp(img, 0., 1.)
    return img
plt.figure(figsize=(8, 8))
for i in range(9):
    img, label = train_set[random.randint(0, len(train_set))]
    img = denorm(img)
    img = img.permute(1, 2, 0)
    ax = plt.subplot(3, 3, i + 1)
    ax.imshow(img.numpy()[:, :, ::-1])
    ax.set_title("label = %d" % label)
    ax.set_xticks([])
    ax.set_yticks([])
plt.show()

In [ ]:
batch_size=32
train_iter=DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_iter=DataLoader(test_set, batch_size=batch_size)

In [ ]:
train_losses=[]
train_accs=[]
test_losses=[]
test_accs=[]

epoches=100
lr = 0.01
loss_fn = nn.CrossEntropyLoss()


def train(model:nn.Module, train_iter, test_iter, num_epochs, loss_fn, mode=0):
    if mode==0: # baseline
        optimizer=optim.SGD(model.fc.parameters(), lr=lr, weight_decay=0.005)
    elif mode==1: # Fine tune Conv5_x
        optimizer=optim.SGD(model.layer4.parameters(), lr=lr, weight_decay=0.005)
    elif mode==2: # Fine tune all the conv layers
        for param in model.named_parameters():
            if 'conv' not in param[0]:
                param[1].requires_grad=False
        optimizer=optim.SGD(model.parameters(), lr=lr, weight_decay=0.05)
    elif mode==3: # Add 2 fc layers
        mlp3=nn.Sequential(nn.Linear(in_features=512, out_features=256, bias=True), 
                           nn.Linear(in_features=256, out_features=128, bias=True), nn.Linear(in_features=128, out_features=100, bias=True))
        for layer in mlp3:
            nn.init.normal_(layer.weight)
            nn.init.normal_(layer.bias)
        model.fc=mlp3

        for param in model.named_parameters():
            param[1].requires_grad=False
        for param in model.fc.parameters():
            param[1].requires_grad=True
        optimizer=optim.SGD(model.parameters(), lr=lr, weight_decay=0.05)
        

    loss=torch.nn.CrossEntropyLoss()
    model=model.to(device)
    print(f"Training on {str(device)}")
    for epoch in range(num_epochs):
        model.train()
        train_loss_sum, test_loss_sum, train_acc_sum, n, batch_count = 0.0, 0.0, 0.0, 0, 0
        for X, y in train_iter:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()  
            y_hat = model(X)
            loss = loss_fn(y_hat, y)
            loss.requires_grad_(True)
            loss.backward()
            optimizer.step()

            train_loss_sum += loss.cpu().item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().cpu().item()
            n += y.shape[0]
            batch_count += 1
        
        with torch.no_grad():
            model.eval()  
            test_acc_sum, n2 = 0.0, 0
            for X, y in test_iter:
                X, y = X.to(device), y.to(device)
                test_loss=loss_fn(model(X), y)
                test_loss_sum+=test_loss.cpu().item()
                test_acc_sum += (model(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
                n2 += y.shape[0]
        
        print('epoch %d, train loss %.4f, test loss %.4f, train acc %.3f, test acc %.3f'
              % (epoch + 1, train_loss_sum / batch_count, test_loss_sum / batch_count, train_acc_sum / n, test_acc_sum / n2))

        train_losses.append(train_loss_sum / batch_count)
        test_losses.append(test_loss_sum / batch_count)
        train_accs.append(train_acc_sum / n)
        test_accs.append(test_acc_sum / n2)


train(model, train_iter, test_iter, epoches, loss_fn, 2)   

In [ ]:
x_axis=[num for num in range(1, epoches+1)]

In [ ]:
plt.plot(x_axis, train_accs, color='r', label='train accuracy')
plt.plot(x_axis, test_accs, color='g', label='test accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(x_axis, train_losses, color='r', label='train loss')
plt.plot(x_axis, test_losses, color='g', label='test loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss')
plt.legend()
plt.show()